### AI 기반 회의 녹취록 요약 경진대회
- https://dacon.io/competitions/official/235813/overview/description


In [ ]:
import pandas as pd
import numpy as np
import json
import os
import random
from tqdm import tqdm
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
from torch import nn

import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore

In [ ]:
seed_everything(11)

> data upload

In [ ]:
TEST_SOURCE = os.path.join("test.json")

In [ ]:
with open(TEST_SOURCE) as f:
    TEST_DATA = json.loads(f.read())

In [ ]:
submission = pd.read_csv("sample_submission.csv")

> json to dataframe

In [ ]:
test = pd.DataFrame(columns=['uid', 'title', 'region', 'context'])
uid = 2000
for data in TEST_DATA:
    for agenda in data['context'].keys():
        context = ''
        for line in data['context'][agenda]:
            context += data['context'][agenda][line]
            context += ' '
        test.loc[uid, 'uid'] = uid
        test.loc[uid, 'title'] = data['title']
        test.loc[uid, 'region'] = data['region']
        test.loc[uid, 'context'] = context[:-1]
        uid += 1

In [ ]:
test['total'] = test.title + ' ' + test.region + ' ' + test.context

In [ ]:
test.head()

,uid,title,region,context,total
2000,2000,제235회 본회의 제1차(2012.06.21.),음성,의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 지금부터 음성군의회 제235회 ...,제235회 본회의 제1차(2012.06.21.) 음성 의석을 정돈하여 주시기 ...
2001,2001,제235회 본회의 제1차(2012.06.21.),음성,"의사일정 제1항, 음성군의회 제235회 제1차 정례회 회기결정의 건을 상정합니다. ...","제235회 본회의 제1차(2012.06.21.) 음성 의사일정 제1항, 음성군..."
2002,2002,제235회 본회의 제1차(2012.06.21.),음성,"의사일정 제2항, 회의록 서명의원 선출의 건을 상정합니다. 제235회 제1차 정례회...","제235회 본회의 제1차(2012.06.21.) 음성 의사일정 제2항, 회의록..."
2003,2003,제235회 본회의 제1차(2012.06.21.),음성,"의사일정 제3항, 예산결산특별위원회 구성의 건을 상정합니다. 예산결산특별위원회 구성...","제235회 본회의 제1차(2012.06.21.) 음성 의사일정 제3항, 예산결..."
2004,2004,제235회 본회의 제1차(2012.06.21.),음성,"의사일정 제4항, 환경분야 현지확인 특별위원회 구성결의안을 상정합니다. 대표발의하신...","제235회 본회의 제1차(2012.06.21.) 음성 의사일정 제4항, 환경분..."


In [ ]:
test.shape

(506, 5)

In [ ]:
test = test[['context']]

In [ ]:
test.head()

,context
2000,의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 지금부터 음성군의회 제235회 ...
2001,"의사일정 제1항, 음성군의회 제235회 제1차 정례회 회기결정의 건을 상정합니다. ..."
2002,"의사일정 제2항, 회의록 서명의원 선출의 건을 상정합니다. 제235회 제1차 정례회..."
2003,"의사일정 제3항, 예산결산특별위원회 구성의 건을 상정합니다. 예산결산특별위원회 구성..."
2004,"의사일정 제4항, 환경분야 현지확인 특별위원회 구성결의안을 상정합니다. 대표발의하신..."


> textrank 사용하기

In [ ]:
!pip install git+https://github.com/lovit/textrank.git

  Cloning https://github.com/lovit/textrank.git to /tmp/pip-req-build-frth873h
  Running command git clone -q https://github.com/lovit/textrank.git /tmp/pip-req-build-frth873h
  Created wheel for textrank: filename=textrank-0.1.2-py3-none-any.whl size=11171 sha256=438c1e691a6c26c6c05da66f4c83156b75eadf4488140b080839d00892d6a752
  Stored in directory: /tmp/pip-ephem-wheel-cache-w60wztxc/wheels/92/77/3d/d596fd63962c68b352c2e39170c2cbe21b9fca35e89aafdf87
Successfully built textrank


In [ ]:
! git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 91 (delta 43), reused 22 (delta 6), pack-reused 0
Unpacking objects: 100% (91/91), done.


In [ ]:
cd Mecab-ko-for-Google-Colab/

/content/Mecab-ko-for-Google-Colab


In [ ]:
%%capture
! bash install_mecab-ko_on_colab190912.sh

In [ ]:
from konlpy.tag import Komoran, Okt, Mecab
from textrank import KeysentenceSummarizer
from textrank import KeywordSummarizer
import textrank

In [ ]:
komoran = Komoran()
def komoran_tokenizer(sent):
    words = komoran.pos(sent, join=True)
    words = [w for w in words if ('/NN' in w or '/XR' in w or '/VA' in w or '/VV' in w)]
    return words

summarizer = KeysentenceSummarizer(
    tokenize = komoran_tokenizer,
    min_sim = 0.3,
    verbose = False
)

In [ ]:
summary = []
for text in tqdm(test['context']):
    keysents = summarizer.summarize(list(test['context']), topk=3) 
    summary.append(keysents)

100%|██████████| 506/506 [6:06:17<00:00, 43.43s/it]


In [ ]:
extractive = []
for n in range(test.shape[0]):
    result = summary[n][0][2] + '\n' + summary[n][1][2] + '\n' + summary[n][2][2]
    extractive.append(result)

In [ ]:
submission['summary'] = extractive
submission.to_csv("summary_text.csv", index=False)
submission

,uid,summary
0,id_2000-AGENDA_1,의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제254회 완주군의회 임시회 제...
1,id_2000-AGENDA_2,의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제254회 완주군의회 임시회 제...
2,id_2000-AGENDA_3,의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제254회 완주군의회 임시회 제...
3,id_2000-AGENDA_4,의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제254회 완주군의회 임시회 제...
4,id_2000-AGENDA_5,의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제254회 완주군의회 임시회 제...
...,...,...
501,id_2096-AGENDA_2,의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제254회 완주군의회 임시회 제...
502,id_2096-AGENDA_3,의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제254회 완주군의회 임시회 제...
503,id_2096-AGENDA_4,의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제254회 완주군의회 임시회 제...
504,id_2096-AGENDA_5,의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제254회 완주군의회 임시회 제...


In [ ]:
submission.to_csv("/content/drive/My Drive/submission_1014.csv")

In [ ]:
submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   uid      506 non-null    object
 1   summary  506 non-null    object
dtypes: object(2)
memory usage: 8.0+ KB


In [ ]:
summary = submission['summary'].drop_duplicates(keep='first')

In [ ]:
summary

0    의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제254회 완주군의회 임시회 제...
Name: summary, dtype: object